In [ ]:
import pandas as pd

#import pyspark

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import round

#conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
#sc = pyspark.SparkContext(conf=conf)
#spark = SparkSession(sc)
#spark = SparkSession.builder.appName('abc').getOrCreate()

In [ ]:
context = SparkContext()

In [ ]:
data = pd.read_csv("insurance.csv")

In [ ]:
sqlcontext = SQLContext(context)

In [ ]:
insurance_df = sqlcontext.createDataFrame(data)

In [ ]:
insurance_df.rdd.getNumPartitions()

In [ ]:
insurance_df.show()

In [ ]:
insurance_df.count()

In [ ]:
insurance_df.select('age', 'sex', 'smoker').show()

In [ ]:
insurance_df.describe().show()

In [ ]:
insurance_df.select('sex').distinct().show()

In [ ]:
insurance_df.crosstab('sex', 'smoker')\
            .select('sex_smoker','yes','no')\
            .show()

In [ ]:
insurance_df.select('age','sex','bmi','region','charges')\
            .where(insurance_df['age'] > 50)\
            .withColumnRenamed('sex','gender')\
            .limit(10)\
            .show()

In [ ]:
insurance_df.groupBy('sex')\
            .count()\
            .show()

In [ ]:
gdc = insurance_df.groupBy('sex', 'smoker')\
            .count()\
            .withColumnRenamed('count','total')

prop = gdc\
        .withColumn('prop', round(gdc.total/insurance_df.count() * 100,2))\
        .drop('total')\
        .where(gdc['sex'] == 'female')\
        .show()

In [ ]:
cbs = insurance_df.groupBy('smoker','sex')\
            .agg({'charges':'avg', 'smoker':'count'})\
            .withColumnRenamed('avg(charges)','avg_charges')\
            .withColumnRenamed('count(smoker)','smoker_cnt')\
            .show()